In [47]:
import pandas as pd
import yfinance as yf
import plotly.express as px
from datetime import datetime, timedelta

In [66]:
tickers = ["ITC.NS", "GODFRYPHLP.NS", "VSTIND.NS"]

In [29]:
start_date = datetime.now() - pd.DateOffset(months=8)
end_date = datetime.now()

In [61]:
df_list = []

In [67]:
for i in range(len(tickers)):
    data = yf.download(tickers[i], start=start_date, end=end_date)
    df_list.append(data)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [72]:
df = pd.concat(df_list, keys=tickers, names=["Ticker", "Date"])
df = df.reset_index()
df.head()

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,ITC.NS,2023-01-09,336.950012,341.299988,336.149994,338.049988,330.763580,10618786
1,ITC.NS,2023-01-10,337.899994,338.399994,332.549988,333.399994,326.213806,6949605
2,ITC.NS,2023-01-11,335.100006,335.100006,331.299988,331.799988,324.648285,4584293
3,ITC.NS,2023-01-12,333.399994,333.600006,328.100006,329.850006,322.740356,6638422
4,ITC.NS,2023-01-13,329.850006,332.250000,326.399994,328.750000,321.664032,14412671


In [53]:
fig = px.line_3d(df,
                 x="Date", y="Close", z="Volume",
                 color="Ticker",
                 labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
                 title="Stock Price over time with Volume on Z axis")
fig.show()

In [75]:
fig = px.line(df,
              x="Date", y="Close",
              color="Ticker",
              labels={'Close':'Closing Price'},
              title="Stock Analysis")
fig.show()


In [56]:
df.head()

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,ITC.NS,2023-01-09,336.950012,341.299988,336.149994,338.049988,330.763580,10618786
1,ITC.NS,2023-01-10,337.899994,338.399994,332.549988,333.399994,326.213806,6949605
2,ITC.NS,2023-01-11,335.100006,335.100006,331.299988,331.799988,324.648285,4584293
3,ITC.NS,2023-01-12,333.399994,333.600006,328.100006,329.850006,322.740326,6638422
4,ITC.NS,2023-01-13,329.850006,332.250000,326.399994,328.750000,321.664032,14412671


In [73]:
df["MA10"] = df.groupby("Ticker")["Close"].rolling(window=10).mean().reset_index(level=0, drop=True)
df["MA30"] = df.groupby("Ticker")["Close"].rolling(window=30).mean().reset_index(level=0, drop=True)


In [77]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA30'], 
                  title=f"{ticker} Moving Averages")
    fig.show()

In [78]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
fig.show()


In [85]:
Godfry = df.loc[df["Ticker"] == "GODFRYPHLP.NS", ['Date', 'Close']].rename(columns={'Close': 'Godfry'})
VSTIND = df.loc[df["Ticker"] == "VSTIND.NS", ['Date', 'Close']].rename(columns={'Close': 'VSTIND'})

In [86]:
df_corr = pd.merge(Godfry, VSTIND, on='Date')

In [88]:
fig = px.scatter(df_corr, x='Godfry', y='VSTIND', 
                 trendline='ols', 
                 title='Correlation between VSTIND and VSTIND')
fig.show()